In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt

from modules.nyc_shapefiles import read_nyc_shapefiles, write_precinct_geojson, create_dataframes

In [2]:
precinct_dict, tract_dict, merged_census_info = read_nyc_shapefiles()
print('Mean number of tracts per precinct: {0}'.format(
    np.mean([len(v['intersections']) for k, v in precinct_dict.items()])
))
print('Mean number of precincts per tract: {0}'.format(
    np.mean([len(v['intersections']) for k, v in tract_dict.items()])
))

Mean number of tracts per precinct: 40.103896103896105
Mean number of precincts per tract: 1.4263279445727484


In [3]:
print('---Keys---')
for k, v in tract_dict['1407'].items():
    print(k, type(v))
print()

tract_dict['1407']

---Keys---
shape_orig <class 'shapely.geometry.polygon.Polygon'>
shape_gps <class 'shapely.geometry.polygon.Polygon'>
intersections <class 'dict'>
properties <class 'collections.OrderedDict'>
census_info <class 'pandas.core.series.Series'>
total_intersection_area <class 'float'>



{'census_info': GEO.id2                                                  36005036100
 GEO.display-label           Census Tract 361, Bronx County, New York
 Population                                                      6099
 Median_Household_Income                                        18799
 Percent_Bachelors_Degree                                         7.8
 Name: 1400000US36005036100, dtype: object,
 'intersections': {'30': {'intersection_area_absolute': 1760194.0239685476,
   'intersection_area_percent_of_tract': 1.0}},
 'properties': OrderedDict([('CTLabel', '361'),
              ('BoroCode', '2'),
              ('BoroName', 'Bronx'),
              ('CT2010', '036100'),
              ('BoroCT2010', '2036100'),
              ('CDEligibil', 'E'),
              ('NTACode', 'BX17'),
              ('NTAName', 'East Tremont'),
              ('PUMA', '3705'),
              ('Shape_Leng', 5701.06188867),
              ('Shape_Area', 1760194.02397)]),
 'shape_gps': <shapely.geometry.pol

In [4]:
print('---Keys---')
for k in precinct_dict['50']:
    print(k)
print()

precinct_dict['50']

---Keys---
shape_orig
shape_gps
intersections
properties
total_population



{'intersections': {'100': {'population_assigned_to_precinct': 1181.8216337416325,
   'population_percent_of_precinct': 0.012841349882442173},
  '104': {'population_assigned_to_precinct': 1927.2504157944732,
   'population_percent_of_precinct': 0.020940974673094751},
  '106': {'population_assigned_to_precinct': 204.34002173250809,
   'population_percent_of_precinct': 0.0022203026574707536},
  '1117': {'population_assigned_to_precinct': 2139.6336508152008,
   'population_percent_of_precinct': 0.02324867268115334},
  '1118': {'population_assigned_to_precinct': 2449.1770533113008,
   'population_percent_of_precinct': 0.026612086433082542},
  '1216': {'population_assigned_to_precinct': 442.23527710202677,
   'population_percent_of_precinct': 0.0048052072846615388},
  '1232': {'population_assigned_to_precinct': 0.00070366883070973338,
   'population_percent_of_precinct': 7.645872607615594e-09},
  '1240': {'population_assigned_to_precinct': 2375.0,
   'population_percent_of_precinct': 0.02580

In [5]:
print(np.sum([d['total_population'] for d in precinct_dict.values()]))
print(np.sum(merged_census_info['Population']))
print(np.sum([d['census_info']['Population'] for d in tract_dict.values()]))

8426743.0
8426743
8426743


In [6]:
# Uncomment to create the geojson shapefile.
# write_precinct_geojson(precinct_dict)

In [7]:
precinct_df, tract_df, intersection_df = create_dataframes(precinct_dict, tract_dict, merged_census_info)

In [8]:
precinct_df

,PrecinctShapefileID,Population,Median_Household_Income,Percent_Bachelors_Degree
Precinct,,,,
1,0,69133.039371,132330.083503,83.828412
5,1,50759.992787,49552.406674,37.904058
6,2,60487.767154,112107.491675,84.791603
7,3,57806.426169,34945.568718,32.632376
9,4,76738.998728,69325.458794,59.913145
10,5,52702.442572,93694.177015,71.278494
13,6,94685.084360,104825.381471,77.011882
14,7,24355.832493,102522.108750,75.036762
17,8,72011.486682,118281.890789,81.870054


In [9]:
tract_df

,GEO.id,GEO.id2,GEO.display-label,Population,Median_Household_Income,Percent_Bachelors_Degree
TractShapefileID,,,,,,
0,1400000US36085000900,36085000900,"Census Tract 9, Richmond County, New York",1860,52500.0,30.3
1,1400000US36061009800,36061009800,"Census Tract 98, New York County, New York",7654,122331.0,75.9
2,1400000US36061010000,36061010000,"Census Tract 100, New York County, New York",1683,112098.0,84.8
3,1400000US36061010200,36061010200,"Census Tract 102, New York County, New York",258,NaN,60.6
4,1400000US36061010400,36061010400,"Census Tract 104, New York County, New York",982,111806.0,72.7
5,1400000US36061011300,36061011300,"Census Tract 113, New York County, New York",126,82361.0,67.0
6,1400000US36061011402,36061011402,"Census Tract 114.02, New York County, New York",2345,151131.0,86.5
7,1400000US36061013000,36061013000,"Census Tract 130, New York County, New York",3277,244375.0,85.4
8,1400000US36061014000,36061014000,"Census Tract 140, New York County, New York",7671,137700.0,84.4


In [10]:
intersection_df[intersection_df['Precinct'] == 1].sort_values('TractPopulationInPrecinct', ascending=False)

,Precinct,TractShapefileId,TractPopulationInPrecinct,PopulationPercentOfPrecinct,GEO.id,GEO.id2,GEO.display-label,Population,Median_Household_Income,Percent_Bachelors_Degree
14,1,2020,8875.000000,1.283757e-01,1400000US36061031703,36061031703,"Census Tract 317.03, New York County, New York",8875,182401.0,87.8
10,1,1905,8058.952230,1.165716e-01,1400000US36061000700,36061000700,"Census Tract 7, New York County, New York",8794,117841.0,86.8
18,1,2111,6887.000000,9.961952e-02,1400000US36061031704,36061031704,"Census Tract 317.04, New York County, New York",6887,152692.0,85.1
13,1,2019,6734.999956,9.742086e-02,1400000US36061002100,36061002100,"Census Tract 21, New York County, New York",6735,NaN,90.6
9,1,1904,6235.185842,9.019111e-02,1400000US36061001502,36061001502,"Census Tract 15.02, New York County, New York",7378,133209.0,91.3
2,1,946,5874.538763,8.497440e-02,1400000US36061001501,36061001501,"Census Tract 15.01, New York County, New York",6502,81867.0,68.0
16,1,2079,5216.000000,7.544873e-02,1400000US36061003900,36061003900,"Census Tract 39, New York County, New York",5216,135250.0,78.0
5,1,1525,5004.999990,7.239664e-02,1400000US36061003300,36061003300,"Census Tract 33, New York County, New York",5005,NaN,83.5
3,1,1312,4387.999351,6.347181e-02,1400000US36061004900,36061004900,"Census Tract 49, New York County, New York",4388,97929.0,80.1
17,1,2110,4374.000000,6.326931e-02,1400000US36061001300,36061001300,"Census Tract 13, New York County, New York",4374,123558.0,88.8
